In [1]:
!pip install unrar
!pip install rarfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import re
import rarfile
import zipfile
import codecs
from sklearn.model_selection import train_test_split

In [3]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPool1D, Dropout, MaxPooling1D, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
! gdown --id 1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg
To: /content/hamshahri.rar
100% 873M/873M [00:09<00:00, 88.6MB/s]


In [5]:
def extract_rar_file(path, extract_path):
  files = []
  with rarfile.RarFile(path, 'r') as rf:
    files = rf.namelist()
    rf.extractall(extract_path)
    
  for file in files:
    path_to_file = fr"{extract_path}\{file}"  

In [6]:
path = 'hamshahri.rar'
extract_path = 'hamshahri'
extract_rar_file(path, extract_path)

In [7]:
def extract_zip_file(path, extract_path):
  with zipfile.ZipFile(path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)

In [8]:
hamshahri_corpus_path = 'hamshahri/hamshahriold/Corpus/Hamshahri-Corpus.zip'
hamshahri_corpus_extract_path = 'hamshahri/hamshahriold/Corpus/hamshahri-corpus'
extract_zip_file(hamshahri_corpus_path, hamshahri_corpus_extract_path)

In [9]:
HAMSHAHRI_CORPUS_PATH = 'hamshahri/hamshahriold/Corpus/hamshahri-corpus/Hamshahri-Corpus.txt'

In [10]:
%%time
with codecs.open(HAMSHAHRI_CORPUS_PATH, "r", "utf-8") as text:
    sentences = []
    pattern = re.compile(".Cat")
    s = ' '
    for line in text:
      for match in re.finditer(pattern, line):
        sentences.append((line,s))
        s = ' '
      s += line         

CPU times: user 58.4 s, sys: 2.27 s, total: 1min
Wall time: 58.5 s


In [11]:
category_punctuation = [re.sub('\.Cat\t', '', line[0]) for line in sentences]
sentences_punctuation = [line[1].replace('\n', '') for line in sentences] 
sentences_punctuation = [line.replace('\r', '') for line in sentences_punctuation] 
sentences_punctuation = [re.sub('\.Cat\s\w+', '', sentence) for sentence in sentences_punctuation]
sentences_punctuation = [re.sub('\.DID\\t\d+S\d+.Date\\t\d+\\\\\d+\\\\\d+', '', sentence) for sentence in sentences_punctuation]

In [12]:
print(sentences_punctuation[1])

  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پايان ناپذير برسيم چون انتهايي در بين ن

In [13]:
corpus = []
for i in range(len(sentences)):
  corpus.append({'text': sentences_punctuation[i], 'category': category_punctuation[i-1]})

In [14]:
corpus[1]

{'category': 'adabh\r\n',
 'text': '  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پاي

In [15]:
df = pd.DataFrame(corpus[1:], columns=['text', 'category'])

In [16]:
df.head()

,text,category
0,در زندگي گروهي از طريق هنر نگاهي به نمايشگاه...,adabh\r\n
1,هنري جهان نمايشگاه هنر در خدمت ديكتاتورها نم...,adabh\r\n
2,نگارخانه ها گالري گلستان: نمايشگاه طرح و ساخ...,adabh\r\n
3,را جدي بگيريم مطالعه اي مقدماتي پيرامون نقش ...,ejtem\r\n
4,سياه و غباري كه سترده نمي شود... اشاره; به ر...,elmfa\r\n


In [17]:
len(df)

165261

### **Export DataFrame to CSV**

In [18]:
df.to_csv('data.csv')

In [24]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r "/content/data.csv" "/content/drive/MyDrive/data.csv"

Mounted at /content/drive


# **Read Dataset**

In [46]:
! gdown --id 1nkz1EwQkxsZtX4TXk6rSCjb2LT7VHW5F

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1nkz1EwQkxsZtX4TXk6rSCjb2LT7VHW5F
To: /content/data.csv
100% 579M/579M [00:03<00:00, 168MB/s]


In [47]:
data = pd.read_csv('data.csv')

In [27]:
data = data.iloc[: , 1:]

In [28]:
data = data[data['text'].str.len() <= 8000]

In [29]:
data.head(3)

,text,category
0,در زندگي گروهي از طريق هنر نگاهي به نمايشگاه...,adabh\r\n
1,هنري جهان نمايشگاه هنر در خدمت ديكتاتورها نم...,adabh\r\n
2,نگارخانه ها گالري گلستان: نمايشگاه طرح و ساخ...,adabh\r\n


In [30]:
len(data)

159790

In [31]:
data = data[data.groupby('category')['category'].transform('count') > 500].copy()

In [32]:
num_unique_categories = data['category'].nunique()

In [33]:
num_unique_categories

31

In [34]:
len(data)

150855

In [35]:
data['category'].value_counts()

siasi\r\n    17181
eqtes\r\n    16555
kharj\r\n    14623
akhar\r\n    13811
vrzsh\r\n    12926
shari\r\n    10100
elmif\r\n     8434
shahr\r\n     8322
shrst\r\n     8237
gungn\r\n     7437
ejtem\r\n     7332
adabh\r\n     4727
havad\r\n     3749
elmfa\r\n     1663
soxan\r\n     1325
aeqts\r\n     1292
polig\r\n     1268
nnaft\r\n     1201
jvarz\r\n     1133
gozar\r\n     1093
maqal\r\n      960
econw\r\n      909
mohit\r\n      903
sporw\r\n      873
artw\r\n       849
nameh\r\n      759
bankb\r\n      739
intep\r\n      679
shora\r\n      639
santj\r\n      604
Cultw\r\n      532
Name: category, dtype: int64

### **Read StopWords**

In [36]:
hamshahri_stopwords_path = 'hamshahri/hamshahriold/Corpus/PersianStopWords.zip'
hamshahri_stopwords_extract_path = 'hamshahri/hamshahriold/Corpus/hamshahri-stopwords'
extract_zip_file(hamshahri_stopwords_path, hamshahri_stopwords_extract_path)
HAMSHAHRI_STOPWORDS_PATH = '/content/hamshahri/hamshahriold/Corpus/hamshahri-stopwords/PersianStopWords.txt'

In [37]:
def read_stop_words(filename):
  with open(filename) as stopwords_file:
    stopwords = stopwords_file.readlines()
  stopwords = [line.replace('\n', '') for line in stopwords] 
  return stopwords

In [38]:
stopwords = read_stop_words(HAMSHAHRI_STOPWORDS_PATH)
print(len(stopwords))

796


In [39]:
# Convert pandas to numpy
np_data = data["text"].values

In [40]:
np_data[0]

'  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پايان ناپذير برسيم چون انتهايي در بين 

**Install hazm library**

In [41]:
# install hazm library
!pip install hazm
from hazm import word_tokenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 5.1 MB/s 
     |████████████████████████████████| 233 kB 51.7 MB/s 
     |████████████████████████████████| 1.4 MB 44.8 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=9f3aa21c60de3e1714a6c14c626f107f2e39dca5daae3dbbd73487ca912a32c2
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154727 sha256=62a1ba47aa8e3e443d1986efada1c9750d78b0c5840488ecca2ed02e55bb6a28
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [42]:
word_tokenized = [word_tokenize(t) for t in np_data]

In [43]:
word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

In [44]:
del word_tokenized
del np_data
del stopwords

In [45]:
corpus = []
for sentence in word_tokenized_filtered:
  s = ' '
  for w in sentence:
    s += w + ' '
  corpus.append(s)  

In [ ]:
del word_tokenized_filtered

In [ ]:
print(corpus[0])

 زندگي گروهي طريق هنر نگاهي نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان باطن ظاهري انسانهاهستند هيبتي حشره تابلوهاي نقاشي نمايشگر گوشه زندگي مورچه ديده ايم سنگين خودرا دوش كشد راه خسته نتيجه آمدن حادثه اتفاقي دستش رها گريز خطر گذارد فرار ساعتي نقطه برمي گردد دوش كشيده نفس زنان عرق ريزان منزل رساند داند راه رفت مورچه استعداد غريزه عملي شروع پايان رساند داند نسل آينده مزاياي استفاده نمايد داستان درازي انتها پاياني ميليونها سال گذشته ميلياردها سال خواهد_گذشت وهمين برنامه ميليونها تجديد گفت سير حيات تكامل پايان ناپذير اسرار جهان پاياني بخواهيم چيز پايان ناپذير برسيم انتهايي نيست جايي رسيم كتاب مورچگان نوشته موريس مترلينگ مورچگان تابلوهاي نقاشي احمد طباطبايي جايگاه ويژه دارند نمايشگاهي آثار هنرمند اول 23 خرداد نگارخانه برگ برپا 36 اثر موضوع نمايش عمومي درآمد طباطبايي مدت 18 سالي نقاشي پردازد موضوع مورچگان كار علت انتخاب سوژه گويد روزهاي زندگي غمگين بودم زماني خواب رفتم مورچه ديدم تصميم گرفتم طرح بكشم كار نظرم جالب آمد روز طراحي ادامه دادم نقاشي مدلها تغييراتي ايجاد ضمن حشره

**One Hot the Y**

In [ ]:
Y = pd.get_dummies(data['category']).values

In [ ]:
Y[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [ ]:
del data

**text tokenizer**

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [ ]:
vocab_size = len(tokenizer.word_index)+1

In [ ]:
data_to_sequences = tokenizer.texts_to_sequences(corpus)

In [ ]:
arr = []
for el in data_to_sequences:
  arr.append(len(el))

In [ ]:
maxlen = max(arr)

In [ ]:
maxlen

795

In [ ]:
del arr
del corpus

In [ ]:
pad_data_to_sequences = pad_sequences(data_to_sequences, maxlen = maxlen)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pad_data_to_sequences, Y, test_size=0.3, random_state=42)

In [ ]:
del data_to_sequences
del pad_data_to_sequences

### **Neural Network**

In [ ]:
embedding_dim = 50

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = maxlen))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(200))
model.add(Dense(num_unique_categories, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 795, 50)           18090050  
                                                                 
 conv1d_1 (Conv1D)           (None, 795, 32)           4832      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 397, 32)          0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 200)               186400    
                                                                 
 dense_1 (Dense)             (None, 30)                6030      
                                                                 
Total params: 18,287,312
Trainable params: 18,287,312
Non-trainable params: 0
__________________________________________

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024)

Epoch 1/10
98/98 [==============================] - 1293s 13s/step - loss: 2.7465 - accuracy: 0.1813 - val_loss: 2.0400 - val_accuracy: 0.4098
Epoch 2/10
98/98 [==============================] - 1482s 15s/step - loss: 1.7470 - accuracy: 0.4895 - val_loss: 1.6368 - val_accuracy: 0.5169
Epoch 3/10
98/98 [==============================] - 1531s 16s/step - loss: 1.5522 - accuracy: 0.5425 - val_loss: 1.5641 - val_accuracy: 0.5417
Epoch 4/10
98/98 [==============================] - 1531s 16s/step - loss: 1.3307 - accuracy: 0.6082 - val_loss: 1.4017 - val_accuracy: 0.5887
Epoch 5/10
98/98 [==============================] - 1573s 16s/step - loss: 1.1596 - accuracy: 0.6537 - val_loss: 1.2371 - val_accuracy: 0.6343
Epoch 6/10
98/98 [==============================] - 1510s 15s/step - loss: 1.0140 - accuracy: 0.6975 - val_loss: 1.1981 - val_accuracy: 0.6433
Epoch 7/10
98/98 [==============================] - 1463s 15s/step - loss: 0.8893 - accuracy: 0.7330 - val_loss: 1.1087 - val_accuracy: 0.6823

In [ ]:
model.save('model.h5')

In [ ]:
# laod model
model = load_model('model.h5')